<a href="https://colab.research.google.com/github/Sabavat-Jayanth-Naik/SMART_ATTENDANCE_SYSTEM/blob/main/smartattendancesystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=e7443be9a9f7bbcfb945d7b71e761c4595d618e5a652aa40a99b94505470a766
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
import os

In [3]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [7]:
# Import necessary libraries
import cv2
import numpy as np
import io
import PIL
from PIL import Image
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import pandas as pd
from datetime import datetime
import time
import os
import face_recognition

# Function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# Function to convert OpenCV Rectangle bounding box image into base64 byte string
def bbox_to_bytes(bbox_array):
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='png')
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
    return bbox_bytes

# Function to perform face recognition
def recognize_face(face_region):
    # Resize the face region for consistency in face recognition
    face_region = cv2.resize(face_region, (100, 100))

    # Find all face locations in the frame
    face_locations = face_recognition.face_locations(face_region)

    if not face_locations:
        return "unknown"

    # Encode the face in the region
    face_encoding = face_recognition.face_encodings(face_region, face_locations)[0]

    # Compare the face encoding with known faces
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

    # If a match is found, return the name
    for i, match in enumerate(matches):
        if match:
            return known_face_names[i]

    # If no match is found, return "unknown"
    return "unknown"

# Initialize Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create folders if they do not exist
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

photos_folder = "/content/drive/MyDrive/smartattendancesystem/photos"
data_folder = "/content/drive/MyDrive/smartattendancesystem/data"
create_folder_if_not_exists(photos_folder)
create_folder_if_not_exists(data_folder)

# List of known faces and their encodings
known_faces = {
    "jayanth": "jayanth1.jpg",
    "ashiq": "ashiq.jpg",
    "harsha": "harsha.jpg"

}

# Load known faces and encodings
known_face_encodings = []
known_face_names = []

for name, image_path in known_faces.items():
    image = face_recognition.load_image_file(os.path.join(photos_folder, image_path))
    encoding = face_recognition.face_encodings(image)[0]
    known_face_encodings.append(encoding)
    known_face_names.append(name)

students_info = {
    "jayanth": {"name": "Jayanth", "branch": "Computer Science", "section": "A"},
    "ashiq": {"name": "Ashiq", "branch": "Electrical", "section": "B"},
    "harsha": {"name": "Harsha", "branch": "Mechanical", "section": "C"},
    "yaswanth": {"name": "Yaswanth", "branch": "Civil", "section": "D"}
}

# Placeholder for the missing video_frame function
def video_frame(label_html, bbox):
    js_reply = eval_js('stream_frame("{}", "{}")'.format(label_html, bbox))
    return js_reply

# Initialize CSV file for attendance
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
csv_file_path = os.path.join(data_folder, f"{current_date}.csv")
columns = ["Name", "Branch", "Section", "Time"]

# Create or append to the Excel sheet
attendance_df = pd.DataFrame(columns=columns)
attendance_df.to_csv(csv_file_path, index=False)

# Function to start webcam stream
def start_webcam():
    js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var imgElement;
        var labelElement;

        var pendingResolve = null;
        var shutdown = false;

        function removeDom() {
            stream.getVideoTracks()[0].stop();
            video.remove();
            div.remove();
            video = null;
            div = null;
            stream = null;
            imgElement = null;
            captureCanvas = null;
            labelElement = null;
        }

        function onAnimationFrame() {
            if (!shutdown) {
                window.requestAnimationFrame(onAnimationFrame);
            }
            if (pendingResolve) {
                var result = "";
                if (!shutdown) {
                    captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
                    result = captureCanvas.toDataURL('image/jpeg', 0.8)
                }
                var lp = pendingResolve;
                pendingResolve = null;
                lp(result);
            }
        }

        async function createDom() {
            if (div !== null) {
                return stream;
            }

            div = document.createElement('div');
            div.style.border = '2px solid black';
            div.style.padding = '3px';
            div.style.width = '100%';
            div.style.maxWidth = '600px';
            document.body.appendChild(div);

            const modelOut = document.createElement('div');
            modelOut.innerHTML = "<span>Status:</span>";
            labelElement = document.createElement('span');
            labelElement.innerText = 'No data';
            labelElement.style.fontWeight = 'bold';
            modelOut.appendChild(labelElement);
            div.appendChild(modelOut);

            video = document.createElement('video');
            video.style.display = 'block';
            video.width = div.clientWidth - 6;
            video.setAttribute('playsinline', '');
            video.onclick = () => { shutdown = true; };
            stream = await navigator.mediaDevices.getUserMedia(
                {video: { facingMode: "environment" }});
            div.appendChild(video);

            imgElement = document.createElement('img');
            imgElement.style.position = 'absolute';
            imgElement.style.zIndex = 1;
            imgElement.onclick = () => { shutdown = true; };
            div.appendChild(imgElement);

            const instruction = document.createElement('div');
            instruction.innerHTML =
                '<span style="color: red; font-weight: bold;">' +
                'When finished, click here or on the video to stop this demo</span>';
            div.appendChild(instruction);
            instruction.onclick = () => { shutdown = true; };

            video.srcObject = stream;
            await video.play();

            captureCanvas = document.createElement('canvas');
            captureCanvas.width = 640;
            captureCanvas.height = 480;
            window.requestAnimationFrame(onAnimationFrame);

            return stream;
        }
        async function stream_frame(label, imgData) {
            if (shutdown) {
                removeDom();
                shutdown = false;
                return '';
            }

            var preCreate = Date.now();
            stream = await createDom();

            var preShow = Date.now();
            if (label != "") {
                labelElement.innerHTML = label;
            }

            if (imgData != "") {
                var videoRect = video.getClientRects()[0];
                imgElement.style.top = videoRect.top + "px";
                imgElement.style.left = videoRect.left + "px";
                imgElement.style.width = videoRect.width + "px";
                imgElement.style.height = videoRect.height + "px";
                imgElement.src = imgData;
            }

            var preCapture = Date.now();
            var result = await new Promise(function(resolve, reject) {
                pendingResolve = resolve;
            });
            shutdown = false;

            return {'create': preShow - preCreate,
                    'show': preCapture - preShow,
                    'capture': Date.now() - preCapture,
                    'img': result};
        }
    ''')

    display(js)

# Function to perform attendance using webcam stream
def perform_attendance():
    # Declare bbox as a global variable
    label_html = ""
    global bbox

    # Start streaming video from the webcam
    start_webcam()

    # Move the current_time assignment outside the loop
    current_time = now.strftime("%H-%M-%S")

    # Use a set to keep track of recognized faces (outside the loop for persistence)
    recognized_faces = set()

    # Loop for capturing frames and performing face recognition
    while True:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break

        # Convert JS response to OpenCV Image
        img = js_to_image(js_reply["img"])

        # Create transparent overlay for bounding box
        bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

        # Convert image to grayscale for face detection
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Get face region coordinates
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Declare face_region outside the loop
        face_region = None

        # Get face bounding box for overlay
        for (x, y, w, h) in faces:
            bbox_array = cv2.rectangle(bbox_array, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Assign value to face_region
            face_region = img[y:y + h, x:x + w]

        bbox_array[:, :, 3] = (bbox_array.max(axis=2) > 0).astype(int) * 255
        # Convert overlay of bbox into bytes
        bbox_bytes = bbox_to_bytes(bbox_array)
        # Update bbox so the next frame gets a new overlay
        bbox = bbox_bytes

        # Check if face_region is not None before performing face recognition
        if face_region is not None:
            # Resize the face region for consistency in face recognition
            face_region = cv2.resize(face_region, (100, 100))

            # Perform face recognition and attendance recording
            name = recognize_face(face_region)

            # Update label_html based on recognition results
            if name != "unknown":
                label_html = f"Recognized: {name}"
            else:
                label_html = "Unknown Face"

            # Record attendance for known students
            if name in students_info and name not in recognized_faces:
                student_info = students_info[name]

                # Append the attendance to the Excel sheet
                attendance_data = {
                    "Name": student_info["name"],
                    "Branch": student_info["branch"],
                    "Section": student_info["section"],
                    "Time": current_time
                }

                # Create or append to the Excel sheet
                attendance_df = pd.DataFrame([attendance_data])
                attendance_df.to_csv(csv_file_path, mode='a', header=False, index=False)

                print(f"Attendance recorded for {name}")

                # Add the recognized face to the set to avoid duplicate recordings
                recognized_faces.add(name)

                # Introduce a delay of 5 seconds before allowing another recording
                time.sleep(5)

        # Introduce a delay of 1 second between frames
        time.sleep(1)

# Perform attendance using webcam stream
bbox = ''
perform_attendance()






<IPython.core.display.Javascript object>

Attendance recorded for jayanth
